In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 28.1 MB/s            


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (48, 48)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


model_1 = create_model_V1()
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3277056   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1

2022-04-14 14:13:05.515888: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 14:13:05.693524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 14:13:05.694765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 14:13:05.696437: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [11]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [12]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        #print("hi")
        #print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator,
                class_weight = weights,
                verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        graph_dir = graph_dir+'/'+model_method.__name__
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [14]:
#model_test(5, 1, create_model_V1, (48,48), 'rgb')

In [15]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 130
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

94781440/94765736 [==============================] - 1s 0us/step


In [16]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 14343 

In [17]:
#model_test(5, 50, transfer_learning_model_resnet_50, (48,48,), 'rgb')

In [18]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 13
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

58900480/58889256 [==============================] - 0s 0us/step


In [19]:
transfer_model_vgg_16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 48, 48, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 3591

In [20]:
#model_test(5, 50, transfer_learning_model_vgg_16, (48,48,), 'rgb')

In [21]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 13
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

SPACE

In [22]:
def transfer_learning_model_resnet_101():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet.ResNet101(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 260
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.4)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_101 = transfer_learning_model_resnet_101()
transfer_model_resnet_101.summary()

171458560/171446536 [==============================] - 4s 0us/step
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem_3 ( (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add_3 (TFOpLambda (None, 48, 48, 3)         0         
_________________________________________________________________
resnet101 (Functional)       (None, 2, 2, 2048)        42658176  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
__________________________________________________________

In [23]:
model_test(5, 30, transfer_learning_model_resnet_101, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.


2022-04-14 14:13:44.841673: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-04-14 14:13:53.849753: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


718/718 - 96s - loss: 2.9943 - accuracy: 0.2565 - val_loss: 1.8786 - val_accuracy: 0.3206

Epoch 00001: val_accuracy improved from -inf to 0.32062, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_1.h5


...

Epoch 2/30
718/718 - 45s - loss: 2.0703 - accuracy: 0.3756 - val_loss: 1.6730 - val_accuracy: 0.3950

Epoch 00002: val_accuracy improved from 0.32306 to 0.39498, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_2.h5
Epoch 3/30
718/718 - 45s - loss: 1.6930 - accuracy: 0.4732 - val_loss: 1.6932 - val_accuracy: 0.4206

Epoch 00003: val_accuracy improved from 0.39498 to 0.42059, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_2.h5
Epoch 4/30
718/718 - 44s - loss: 1.3818 - accuracy: 0.5613 - val_loss: 1.6798 - val_accuracy: 0.4263

Epoch 00004: val_accuracy improved from 0.42059 to 0.42633, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_2.h5
Epoch 5/30
718/718 - 45s - loss: 1.1232 - accuracy: 0.6430 - val_loss: 1.7266 - val_accuracy: 0.4

225/225 [==============================] - 18s 81ms/step - loss: 2.4152 - accuracy: 0.5575
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 55s - loss: 2.9854 - accuracy: 0.2662 - val_loss: 1.8783 - val_accuracy: 0.3227

Epoch 00001: val_accuracy improved from -inf to 0.32271, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_3.h5
Epoch 2/30
718/718 - 45s - loss: 2.0536 - accuracy: 0.3784 - val_loss: 1.9945 - val_accuracy: 0.3069

Epoch 00002: val_accuracy did not improve from 0.32271
Epoch 3/30
718/718 - 45s - loss: 1.6809 - accuracy: 0.4756 - val_loss: 1.6062 - val_accuracy: 0.4387

Epoch 00003: val_accuracy improved from 0.32271 to 0.43870, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_3.h5
Epoch 4/30
718/718 - 46s - los

225/225 [==============================] - 12s 52ms/step - loss: 2.3154 - accuracy: 0.5584
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 55s - loss: 2.9450 - accuracy: 0.2734 - val_loss: 1.7956 - val_accuracy: 0.3471

Epoch 00001: val_accuracy improved from -inf to 0.34709, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_4.h5
Epoch 2/30
718/718 - 49s - loss: 2.0209 - accuracy: 0.3933 - val_loss: 1.7241 - val_accuracy: 0.3802

Epoch 00002: val_accuracy improved from 0.34709 to 0.38018, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_4.h5
Epoch 3/30
718/718 - 45s - loss: 1.6635 - accuracy: 0.4791 - val_loss: 1.7637 - val_accuracy: 0.4100

Epoch 00003: val_accuracy improved from 0.38018 to 0.40996, saving model to /kaggle/w

225/225 [==============================] - 17s 76ms/step - loss: 2.6420 - accuracy: 0.5529
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 56s - loss: 2.9297 - accuracy: 0.2629 - val_loss: 2.0719 - val_accuracy: 0.2689

Epoch 00001: val_accuracy improved from -inf to 0.26894, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_5.h5
Epoch 2/30
718/718 - 45s - loss: 2.0193 - accuracy: 0.3830 - val_loss: 1.8622 - val_accuracy: 0.3278

Epoch 00002: val_accuracy improved from 0.26894 to 0.32782, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_101_30epochs/transfer_learning_model_resnet_101_model_5.h5
Epoch 3/30
718/718 - 45s - loss: 1.6352 - accuracy: 0.4872 - val_loss: 1.5557 - val_accuracy: 0.4445

Epoch 00003: val_accuracy improved from 0.32782 to 0.44452, saving model to /kaggle/w

225/225 [==============================] - 19s 82ms/step - loss: 2.4183 - accuracy: 0.5662


In [24]:
def transfer_learning_model_densenet_169():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.densenet.preprocess_input(inputs)
  base_model = tf.keras.applications.densenet.DenseNet169(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 360
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.4)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_densenet_169 = transfer_learning_model_densenet_169()
transfer_model_densenet_169.summary()

51888128/51877672 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 48, 48, 3)         0         
_________________________________________________________________
densenet169 (Functional)     (None, 1, 1, 1664)        12642880  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1664)              0         
______________________________________________________________

In [25]:
model_test(5, 30, transfer_learning_model_densenet_169, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 78s - loss: 2.8789 - accuracy: 0.2346 - val_loss: 2.1506 - val_accuracy: 0.2717

Epoch 00001: val_accuracy improved from -inf to 0.27168, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_1.h5
Epoch 2/30
718/718 - 61s - loss: 2.2718 - accuracy: 0.3275 - val_loss: 1.9829 - val_accuracy: 0.3372

Epoch 00002: val_accuracy improved from 0.27168 to 0.33716, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_1.h5
Epoch 3/30
718/718 - 62s - loss: 1.9621 - accuracy: 0.4120 - val_loss: 1.8053 - val_accuracy: 0.4056

Epoch 00003: val_accuracy improved from 0.33716 to 0.40561, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_lear

225/225 [==============================] - 20s 88ms/step - loss: 2.1348 - accuracy: 0.5607
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 77s - loss: 2.9396 - accuracy: 0.2407 - val_loss: 2.1543 - val_accuracy: 0.2712

Epoch 00001: val_accuracy improved from -inf to 0.27116, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_2.h5
Epoch 2/30
718/718 - 62s - loss: 2.2699 - accuracy: 0.3340 - val_loss: 1.8214 - val_accuracy: 0.3748

Epoch 00002: val_accuracy improved from 0.27116 to 0.37478, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_2.h5
Epoch 3/30
718/718 - 62s - loss: 1.9795 - accuracy: 0.4104 - val_loss: 1.7712 - val_accuracy: 0.4140

Epoch 00003: val_accuracy improved from 0.37478 to 0.41397, saving model to /

225/225 [==============================] - 12s 54ms/step - loss: 2.3046 - accuracy: 0.5548
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 83s - loss: 2.8486 - accuracy: 0.2420 - val_loss: 2.2789 - val_accuracy: 0.2564

Epoch 00001: val_accuracy improved from -inf to 0.25636, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_3.h5
Epoch 2/30
718/718 - 64s - loss: 2.2343 - accuracy: 0.3403 - val_loss: 1.8171 - val_accuracy: 0.3737

Epoch 00002: val_accuracy improved from 0.25636 to 0.37374, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_3.h5
Epoch 3/30
718/718 - 65s - loss: 1.9520 - accuracy: 0.4180 - val_loss: 1.7492 - val_accuracy: 0.4361

Epoch 00003: val_accuracy improved from 0.37374 to 0.43608, saving model to /

225/225 [==============================] - 12s 52ms/step - loss: 2.4076 - accuracy: 0.5548
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 83s - loss: 2.8761 - accuracy: 0.2429 - val_loss: 2.1251 - val_accuracy: 0.2731

Epoch 00001: val_accuracy improved from -inf to 0.27308, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_4.h5
Epoch 2/30
718/718 - 67s - loss: 2.2641 - accuracy: 0.3364 - val_loss: 1.9234 - val_accuracy: 0.3424

Epoch 00002: val_accuracy improved from 0.27308 to 0.34239, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_4.h5
Epoch 3/30
718/718 - 64s - loss: 1.9696 - accuracy: 0.4081 - val_loss: 1.8882 - val_accuracy: 0.3844

Epoch 00003: val_accuracy improved from 0.34239 to 0.38436, saving model to /

225/225 [==============================] - 13s 59ms/step - loss: 2.2123 - accuracy: 0.5584
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 84s - loss: 2.9655 - accuracy: 0.2356 - val_loss: 2.2223 - val_accuracy: 0.2729

Epoch 00001: val_accuracy improved from -inf to 0.27295, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_5.h5
Epoch 2/30
718/718 - 64s - loss: 2.2834 - accuracy: 0.3285 - val_loss: 1.9426 - val_accuracy: 0.3332

Epoch 00002: val_accuracy improved from 0.27295 to 0.33322, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_densenet_169_30epochs/transfer_learning_model_densenet_169_model_5.h5
Epoch 3/30
718/718 - 65s - loss: 2.0020 - accuracy: 0.4002 - val_loss: 1.7576 - val_accuracy: 0.4062

Epoch 00003: val_accuracy improved from 0.33322 to 0.40620, saving model to /

225/225 [==============================] - 12s 52ms/step - loss: 2.2338 - accuracy: 0.5581
